In [1]:
# import os
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

# os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
#초기에 넣어 줄 랜덤 넘버
batch_size = 1000
input_scale = 10
bins = 201
iteration = 1000000
cost_graph=[]
iteration_graph=[]
input = tf.random_normal([batch_size,input_scale])

In [3]:
#test용 input, potential
potential = tf.constant(0.0, shape = [batch_size,bins])

In [4]:
#웨이트, 바이어스
'''간단하게 기존의 프로그램과 차이점은 기존의 프로그램에 두번째 스테이트를 쌓는 다는 점이다.'''
initializer = tf.contrib.layers.xavier_initializer()
# weight = tf.zeros([2, input_scale, bins], dtype = tf.float32)
# weight = tf.zeros([input_scale, bins], dtype = tf.float32)
weight1 = tf.Variable(initializer([input_scale, bins]), dtype = tf.float32)
bias1 = tf.Variable(initializer([bins]), dtype = tf.float32)
weight2 = tf.Variable(initializer([input_scale, bins]), dtype = tf.float32)
bias2 = tf.Variable(initializer([bins]), dtype = tf.float32)

In [5]:
w1_loss, b1_loss, w2_loss, b2_loss = tf.nn.l2_loss(weight1), tf.nn.l2_loss(bias1), tf.nn.l2_loss(weight2), tf.nn.l2_loss(bias2)

In [6]:
binsnorm = tf.sqrt(tf.cast(bins, tf.float32))
#프사이 1
psi_1 = tf.matmul(input, weight1) + bias1
psi_1 = tf.divide(psi_1,tf.sqrt(tf.reduce_mean(tf.square(psi_1), axis = 1, keepdims = True)))/binsnorm

## 에너지 1
zeroten = tf.zeros([batch_size, 1], tf.float32)
psil_1 = tf.concat([psi_1[:,1:],zeroten],1)
psir_1 = tf.concat([zeroten,psi_1[:,:-1]],1)
energy_1 = tf.reduce_mean(tf.subtract(tf.multiply(tf.square(psi_1*binsnorm),tf.add(potential,1.*bins*bins)),tf.multiply(tf.multiply(tf.add(psil_1*binsnorm,psir_1*binsnorm),psi_1*binsnorm),0.5*bins*bins)), axis=-1, keep_dims=True)

## 그람슈미츠 직교화
'''그람슈미츠 직교화를 통해 프사이1과 orthonormal한 프사이2를 만드는 과정이다.'''
psi_2 = tf.matmul(input, weight2) + bias2
dotprod_1_2 = tf.multiply(tf.reduce_sum(tf.multiply(psi_1, psi_2), axis = 1, keepdims = True), psi_1)
psi_2 = psi_2-dotprod_1_2
psi_2 = tf.divide(psi_2,tf.sqrt(tf.reduce_mean(tf.square(psi_2), axis = 1, keepdims = True)))/binsnorm


psil_2 = tf.concat([psi_2[:,1:],zeroten],1)
psir_2 = tf.concat([zeroten,psi_2[:,:-1]],1)
energy_2 = tf.reduce_mean(tf.subtract(tf.multiply(tf.square(psi_2*binsnorm),tf.add(potential,1.*bins*bins)),tf.multiply(tf.multiply(tf.add(psil_2*binsnorm,psir_2*binsnorm),psi_2*binsnorm),0.5*bins*bins)), axis=-1, keep_dims=True)

## Leanier combination
'''사실 이 계수는 별로 중요하지 않다. 왜냐하면 두 파동함수가 직교하기 때문에
첫번째 프사이가 그라운드 스테이트로 간 후,
두번째 프사이가 그라운드 스테이트로 가기 위해서 무조건 첫번째 들뜬 상태로 간다'''
C=1/tf.sqrt(2.)
C_1 = C*tf.ones(shape=[batch_size, 1])
C_2 = C*tf.ones(shape=[batch_size, 1])
psi = tf.add(tf.multiply(C_1, psi_1), tf.multiply(C_2, psi_2))

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [7]:
total_energy = (C_1**2)*energy_1+(C_2**2)*energy_2
# total_energy = energy_1
# cost = tf.reduce_mean(total_energy)
cost = tf.reduce_mean(total_energy)+0.02*(w1_loss+w2_loss)

In [9]:
'''여기선 varlist라는 개념을 사용했는데 이는 텐서플로가 변수로 인식하는 부분을 의미한다.
첫번째 학습에서는 웨이트1과 바이어스1만이 텐서플로의 변수이고
두번째 학습에서는 웨이트2와 바이어스2만이 변수로 작용한다.'''
train = tf.train.AdamOptimizer(learning_rate=0.001).minimize(energy_1, var_list=[weight1, bias1])
train2 = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost, var_list=[weight2, bias2])

In [ ]:
'''생각보다 프로그램이 무거워져서 GPU를 사용해야 빠르게 돌아간다.'''
# If you want to use CPU
config = tf.ConfigProto(device_count = {'GPU': 0})

sess = tf.Session(config=config)

# If you want to use GPU
# config = tf.ConfigProto()
# config.gpu_options.allow_growth = True

# sess = tf.Session(config=config)

In [ ]:
init = tf.global_variables_initializer()
sess.run(init)

In [ ]:
'''기존으 프로그램을 통해 이터레이션이 약 20000정도 진행되면 충분히 그라운드 스테이트에 도달한다는 점을
착안하여, 30000번 쯤을 기준으로 학습1에서 학습2로 넘어가게 설정했다.
파란색은 그라운드, 초록색은 퍼스트, 빨간색은 이 둘의 합을 나타내는데
퍼스트가 한쪽으로 치우친 두가지형태가 있기 때문에 둘의 합도 한쪽으로 치우치게 된다.
그림에서 퍼스트가 대칭인 이유는 제곱이기 때문.'''
for i in range (iteration):  
    if i <30000:
        sess.run(train)
    else :
        sess.run(train2)

    if i % 1000 == 0:
        psi_total,psi_1v,psi_2v,cost_now,w1_loss_v,b1_loss_v,w2_loss_v,b2_loss_v, energy1_now, energy2_now = sess.run([psi, psi_1, psi_2, cost, w1_loss, b1_loss, w2_loss, b2_loss, tf.reduce_mean(energy_1), tf.reduce_mean(energy_2)])
        prob_total,prob_1v,prob_2v = psi_total**2, psi_1v**2, psi_2v**2  
        print("iteration=%d, cost=%f, energy1=%f, energy2=%f" % (i, cost_now, energy1_now, energy2_now))
        print("W1_loss=%f, B1_loss=%f, W2_loss=%f, B2_loss=%f" % (w1_loss_v, b1_loss_v, w2_loss_v, b2_loss_v))
        if i < 30000:
            plt.plot(prob_1v[0], '-b')
        else:
            plt.plot(prob_1v[0], '-b')
            plt.plot(prob_total[0], '-r')
            plt.plot(prob_2v[0], '-g')
        plt.show()
        